In [1]:
                        ##-- BASIC EXPLATORY DATA ANALYSIS - 2 --##
# Importing libraries

import pandas as pd
import getpass
import math
import numpy as np
from tqdm import trange
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest


if getpass.getuser() == "asus":
    file_path = "C:/Users/asus/Desktop/trendyol_data_v2/"
    
if getpass.getuser() == "your computers name":
    file_path = "C:/Users/..desiredlocation..."

In [2]:
reviews = pd.read_csv(file_path + 'reviews_v2.csv') #6m rows 

In [3]:
reviews.head()

,product_content_id,rate,comment,supplier_id,new_rate,is_return_mentioned
0,35244093,5,75c göğüse S beden aldım spor için daha da sar...,968,5,0
1,97842293,5,fiyatı yüksek ama çok memnunum,367,5,0
2,86701595,5,yorumlara ve beden tablosuna bakarak 1 beden b...,191874,5,0
3,107237246,3,çok kısa iade ettim. sıradan geldi,968,3,1
4,45019480,5,Renkler daha canlı. Hoş,968,5,0


In [3]:
main_data = pd.read_csv(file_path + 'merged_transactions_v6.csv')

In [7]:
main_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772659 entries, 0 to 10772658
Data columns (total 44 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   is_elite_user         int64  
 2   supplier_id           int64  
 3   order_parent_id       int64  
 4   product_content_id    int64  
 5   product_variant_id    int64  
 6   discounted_price      float64
 7   ship_cost             float64
 8   is_wallet_trx         int64  
 9   is_saved_card_trx     int64  
 10  is_returned           float64
 11  coupon_used           int64  
 12  is_different_sizes    int64  
 13  is_discounted         int64  
 14  promotion_percent     float64
 15  is_shipcost_incurred  int64  
 16  shipping_percent      float64
 17  Erkek                 int64  
 18  GenderlessItem        int64  
 19  Kadın                 int64  
 20  Kadın_Kız             int64  
 21  Unisex                int64  
 22  Altın                 int64  
 23  Bej  

In [6]:
main_data.category_name.unique() #there are too many categories, consuming my memory a lot. Even if one-hot encode them, there are too many of them.
#I think I am going to drop this column in order to save memory, at the expense of possible predictive power.

array(['T-Shirt', 'Polo Yaka T-shirt', 'Elbise', 'Şort & Bermuda',
       'Sweatshirt', 'Eşofman Takımı', 'Alt-Üst Takım', 'Pijama Takımı',
       'Kazak', 'Pantolon', 'Bluz', 'Ceket', 'Eşofman Altı', 'Gömlek',
       'Jeans', 'Tayt', 'Atlet', 'Sütyen', 'Hastane Çıkışları', 'Mayo',
       'Bikini Takımı', 'Bikini Altı', 'Tulum', 'Pijama Altı', 'Sabahlık',
       'Pijama Üstü', 'Çorap', 'Babydoll', 'Etek', 'Trençkot', 'Hırka',
       'Boxer', 'Uyku Tulumu', 'Body', 'Hamile T-Shirt', 'Külot', 'Pareo',
       'Bikini Üstü', 'Fantezi Babydoll', 'Mont', 'Büstiyer',
       'Tesettür Tunik', 'Kimono&Kaftan', 'Büyük Beden Pijama Takımı',
       'Fantezi Aksesuar', 'Bebek Body & Zıbın', 'Bebek Takımları',
       'Gecelik', 'Büyük Beden Bluz', 'Büyük Beden Elbise',
       'Fantezi Gecelik', 'Abiye & Mezuniyet Elbisesi', 'Deniz Şortu',
       'Yağmurluk&Rüzgarlık', 'Spor Sütyeni', 'Eşarp', 'Tesettür Mayo',
       'Tunik', 'Hamile Gecelik', 'Hamile Külodu', 'Emzirme Sütyeni',
       'İç çamaşırı t

In [4]:
main_data = main_data.drop(['category_name'],axis=1)
main_data.info(memory_usage='deep') #now it is more manageable (3.6 gb)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772659 entries, 0 to 10772658
Data columns (total 45 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   is_elite_user         int64  
 2   supplier_id           int64  
 3   order_parent_id       int64  
 4   product_content_id    int64  
 5   product_variant_id    int64  
 6   discounted_price      float64
 7   ship_cost             float64
 8   is_wallet_trx         int64  
 9   is_saved_card_trx     int64  
 10  is_returned           float64
 11  coupon_used           int64  
 12  is_different_sizes    int64  
 13  is_discounted         int64  
 14  promotion_percent     float64
 15  is_shipcost_incurred  int64  
 16  shipping_percent      float64
 17  product_id            int64  
 18  Erkek                 int64  
 19  GenderlessItem        int64  
 20  Kadın                 int64  
 21  Kadın_Kız             int64  
 22  Unisex                int64  
 23  Altın

In [5]:
reviews = reviews.drop(['comment'],axis=1)

In [7]:
reviews.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6176841 entries, 0 to 6176840
Data columns (total 5 columns):
 #   Column               Dtype
---  ------               -----
 0   product_content_id   int64
 1   rate                 int64
 2   supplier_id          int64
 3   new_rate             int64
 4   is_return_mentioned  int64
dtypes: int64(5)
memory usage: 235.6 MB


In [9]:
reviews.is_return_mentioned.sum()

479541

In [10]:
reviews.head()

,product_content_id,rate,supplier_id,new_rate,is_return_mentioned
0,35244093,5,968,5,0
1,97842293,5,367,5,0
2,86701595,5,191874,5,0
3,107237246,3,968,3,1
4,45019480,5,968,5,0


Return probability of an item might depend on both the supplier and the product itself. Considering the fact that we cannot use
'supplier_id' in test data, it will be better if we could somehow focus on the product_content_id's contribution to the return probability.

In [8]:
len(main_data.product_content_id.unique())

684112

In [10]:
len(reviews.product_content_id.unique())

462323

In [9]:
pd.set_option("display.max_columns", None)
main_data.head()

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,product_id,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_probability
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,82687030,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,30551316,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,30961805,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
3,3941815,0,163856,595758968,36888399,65613254,49.90,10.67,0,1,0.0,0,0,0,0.000000,1,0.213828,30961856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN
4,15996105,1,968,595758987,96930601,161408176,71.99,9.40,0,0,0.0,0,0,1,0.200022,1,0.130574,82735052,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN


In [6]:
main_data = main_data.drop(['product_id'],axis=1)
main_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772659 entries, 0 to 10772658
Data columns (total 44 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   is_elite_user         int64  
 2   supplier_id           int64  
 3   order_parent_id       int64  
 4   product_content_id    int64  
 5   product_variant_id    int64  
 6   discounted_price      float64
 7   ship_cost             float64
 8   is_wallet_trx         int64  
 9   is_saved_card_trx     int64  
 10  is_returned           float64
 11  coupon_used           int64  
 12  is_different_sizes    int64  
 13  is_discounted         int64  
 14  promotion_percent     float64
 15  is_shipcost_incurred  int64  
 16  shipping_percent      float64
 17  Erkek                 int64  
 18  GenderlessItem        int64  
 19  Kadın                 int64  
 20  Kadın_Kız             int64  
 21  Unisex                int64  
 22  Altın                 int64  
 23  Bej  

In [8]:
main_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772659 entries, 0 to 10772658
Data columns (total 44 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   is_elite_user         int64  
 2   supplier_id           int64  
 3   order_parent_id       int64  
 4   product_content_id    int64  
 5   product_variant_id    int64  
 6   discounted_price      float64
 7   ship_cost             float64
 8   is_wallet_trx         int64  
 9   is_saved_card_trx     int64  
 10  is_returned           float64
 11  coupon_used           int64  
 12  is_different_sizes    int64  
 13  is_discounted         int64  
 14  promotion_percent     float64
 15  is_shipcost_incurred  int64  
 16  shipping_percent      float64
 17  Erkek                 int64  
 18  GenderlessItem        int64  
 19  Kadın                 int64  
 20  Kadın_Kız             int64  
 21  Unisex                int64  
 22  Altın                 int64  
 23  Bej  

In [11]:
#from tqdm import tqdm

In [24]:
#main_data_v2 = pd.merge(main_data,reviews[['product_content_id','supplier_id','new_rate','is_return_mentioned']],how='left',on=['product_content_id','supplier_id'])

In [12]:
reviews.head()

,product_content_id,rate,supplier_id,new_rate,is_return_mentioned
0,35244093,5,968,5,0
1,97842293,5,367,5,0
2,86701595,5,191874,5,0
3,107237246,3,968,3,1
4,45019480,5,968,5,0


In [10]:
reviews_v2 = pd.DataFrame(reviews.groupby(['product_content_id','supplier_id']).agg(
    return_probability_2 = pd.NamedAgg(column='is_return_mentioned',aggfunc='mean')).reset_index())

In [11]:
reviews_v2.head()

,product_content_id,supplier_id,return_probability_2
0,48921,398,0.0
1,48921,117031,1.0
2,63682,153388,0.0
3,63785,153388,0.0
4,82578,398,0.0


In [15]:
#reviews_v2['prod_supp_id'] = reviews_v2['product_content_id'].map(str) + "|" + reviews_v2['supplier_id'].map(str)

In [16]:
reviews_v2 = reviews_v2.drop(['product_content_id','supplier_id'],axis=1)

In [12]:
reviews_v2.head()

,product_content_id,supplier_id,return_probability_2
0,48921,398,0.0
1,48921,117031,1.0
2,63682,153388,0.0
3,63785,153388,0.0
4,82578,398,0.0


In [13]:
reviews_v2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564283 entries, 0 to 564282
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   product_content_id    564283 non-null  int64  
 1   supplier_id           564283 non-null  int64  
 2   return_probability_2  564283 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 12.9 MB


In [29]:
#main_data['prod_supp_id'] = main_data['product_content_id'].map(str) + "|" + main_data['supplier_id'].map(str)

In [14]:
main_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772659 entries, 0 to 10772658
Data columns (total 44 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   is_elite_user         int64  
 2   supplier_id           int64  
 3   order_parent_id       int64  
 4   product_content_id    int64  
 5   product_variant_id    int64  
 6   discounted_price      float64
 7   ship_cost             float64
 8   is_wallet_trx         int64  
 9   is_saved_card_trx     int64  
 10  is_returned           float64
 11  coupon_used           int64  
 12  is_different_sizes    int64  
 13  is_discounted         int64  
 14  promotion_percent     float64
 15  is_shipcost_incurred  int64  
 16  shipping_percent      float64
 17  Erkek                 int64  
 18  GenderlessItem        int64  
 19  Kadın                 int64  
 20  Kadın_Kız             int64  
 21  Unisex                int64  
 22  Altın                 int64  
 23  Bej  

In [40]:
main_data.head()

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,is_returned,coupon_used,is_different_sizes,is_discounted,promotion_percent,is_shipcost_incurred,shipping_percent,Erkek,GenderlessItem,Kadın,Kadın_Kız,Unisex,Altın,Bej,Beyaz,Bordo,Ekru,Gri,Gümüş,Haki,Kahverengi,Kırmızı,Lacivert,Mavi,Metalik,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_probability,prod_supp_id
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,0.0,0,0,1,0.349937,1,0.170966,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,95503167|4662
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,0.0,0,0,1,0.538515,1,0.156745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,36313798|163
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,0.0,0,0,1,0.166694,1,0.213443,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,36888320|157306
3,3941815,0,163856,595758968,36888399,65613254,49.90,10.67,0,1,0.0,0,0,0,0.000000,1,0.213828,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,36888399|163856
4,15996105,1,968,595758987,96930601,161408176,71.99,9.40,0,0,0.0,0,0,1,0.200022,1,0.130574,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,96930601|968


In [31]:
tqdm.pandas()

In [37]:
reviews_v2.head()

,return_probability_2,prod_supp_id
0,0.0,48921|398
1,1.0,48921|117031
2,0.0,63682|153388
3,0.0,63785|153388
4,0.0,82578|398


In [42]:
#main_data = main_data.drop(['prod_supp_id'],axis=1)

In [16]:
reviews_v2.head()

,product_content_id,supplier_id,return_probability_2
0,48921,398,0.0
1,48921,117031,1.0
2,63682,153388,0.0
3,63785,153388,0.0
4,82578,398,0.0


In [17]:
main_data_v2 = pd.merge(main_data,reviews_v2,how='left',on=['product_content_id','supplier_id'])

In [19]:
main_data_v2.head()

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,...,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_probability,return_probability_2
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,...,0,0,0,0,0,0,0,0,NaN,0.000000
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,...,0,0,0,0,0,0,0,1,NaN,0.047619
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,...,0,0,0,0,0,0,0,0,NaN,NaN
3,3941815,0,163856,595758968,36888399,65613254,49.90,10.67,0,1,...,1,0,0,0,0,0,0,0,NaN,NaN
4,15996105,1,968,595758987,96930601,161408176,71.99,9.40,0,0,...,0,0,0,0,0,0,0,0,NaN,0.083333


In [20]:
main_data_v2 = main_data_v2.fillna(0)

In [21]:
main_data_v2.head()

,user_id,is_elite_user,supplier_id,order_parent_id,product_content_id,product_variant_id,discounted_price,ship_cost,is_wallet_trx,is_saved_card_trx,...,Mor,Multicolor,Pembe,Sarı,Siyah,Turkuaz,Turuncu,Yeşil,return_probability,return_probability_2
0,3941815,0,4662,595758968,95503167,159147197,51.94,8.88,0,1,...,0,0,0,0,0,0,0,0,0.0,0.000000
1,3941815,0,163,595758968,36313798,64622938,59.97,9.40,0,1,...,0,0,0,0,0,0,0,1,0.0,0.047619
2,3941815,0,157306,595758968,36888320,65612791,49.99,10.67,0,1,...,0,0,0,0,0,0,0,0,0.0,0.000000
3,3941815,0,163856,595758968,36888399,65613254,49.90,10.67,0,1,...,1,0,0,0,0,0,0,0,0.0,0.000000
4,15996105,1,968,595758987,96930601,161408176,71.99,9.40,0,0,...,0,0,0,0,0,0,0,0,0.0,0.083333


In [22]:
main_data_v2.to_csv(file_path+'merged_transactions_v7.csv')

We continue from exploratory_da_v3. The data gets larger and larger after every join, that why I backup my files and continue from new notebooks.